# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [159]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [160]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [161]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [187]:
df_prestige_binary = pd.get_dummies(df['prestige'], prefix = 'prestige')
df_prestige_binary.columns = ['prestige_1','prestige_2', 'prestige_3','prestige_4']
df_new = pd.concat([df, df_prestige_binary], axis=1)
df_new.head()

,admit,gre,gpa,prestige,prestige_1,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,3.0,0.0,0.0,1.0,0.0
1,1,660.0,3.67,3.0,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,1.0,0.0,0.0,0.0
3,1,640.0,3.19,4.0,0.0,0.0,0.0,1.0
4,0,520.0,2.93,4.0,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We will need, one for each of the dummified prestige variables

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [189]:
cols_to_keep = ['admit', 'gre', 'gpa',]
handCalc = df[cols_to_keep].join(df_prestige_binary.ix[:, 'prestige_1':])
handCalc 
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         0.0         1.0         0.0
1      1  660.0  3.67         0.0         0.0         1.0         0.0
2      1  800.0  4.00         1.0         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         0.0         1.0


In [164]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
admit_pres1_cross = pd.crosstab(handCalc.admit, handCalc.prestige_1)
print 'Prestige 1'
print
print admit_pres1_cross

Prestige 1

prestige_1  0.0  1.0
admit               
0           243   28
1            93   33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [165]:
prob_adm = float(33)/(33+28)
odds_ratio_adm =  prob_adm / (1 - prob_adm)

print 'Prob. Admit (Prestige 1):', prob_adm
print 'Odds of Admit (Prestige 1):', odds_ratio_adm

Prob. Admit (Prestige 1): 0.540983606557
Odds of Admit (Prestige 1): 1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [166]:
prob_admother = float(93)/(93+243)
odds_ratio_admother = prob_admother / (1 - prob_admother)

print 'Probs. Admit (Prestige 2-4):', prob_admother
print 'Odds of Admit (Prestige 2-4):', odds_ratio_admother

Probs. Admit (Prestige 2-4): 0.276785714286
Odds of Admit (Prestige 2-4): 0.382716049383


#### 3.3 Calculate the odds ratio

In [167]:
print 'Odds Ratio'
print '\n'
print odds_ratio_adm / odds_ratio_admother

Odds Ratio


3.07949308756


#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being admitted, if you went highest prestige schools, is 3.08 times larger than if you went to a lower prestige school. 

#### 3.5 Print the cross tab for prestige_4

In [168]:
admit_pres4_cross = pd.crosstab(handCalc.admit, handCalc.prestige_4)
print 'Prestige 4'
print
print admit_pres4_cross

Prestige 4

prestige_4  0.0  1.0
admit               
0           216   55
1           114   12


#### 3.6 Calculate the OR 

In [169]:
#Prestige 4 admitted
prob_adm_2 = float(12)/(12+55)
odds_ratio_adm_2 =  prob_adm_2 / (1 - prob_adm_2)

#Prestige 1-3 admitted
prob_admother_2 = float(114)/(114+216)
odds_ratio_admother_2 =  prob_admother_2 / (1 - prob_admother_2)

print 'Odds Ratio'
print odds_ratio_adm_2 / odds_ratio_admother_2

Odds Ratio
0.413397129187


#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted from the lowest prestige school is lower than if you went to a school of a higher prestige. 

## Part 4. Analysis

In [170]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(df_prestige_binary.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         1.0         0.0
1      1  660.0  3.67         0.0         1.0         0.0
2      1  800.0  4.00         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [171]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [227]:
train_cols = data.columns[1:]
train_data = data[train_cols]
y = data['admit']
#train_cols.head()

In [228]:
train_data.head()

,gre,gpa,prestige_2,prestige_3,prestige_4,intercept
0,380.0,3.61,0.0,1.0,0.0,1.0
1,660.0,3.67,0.0,1.0,0.0,1.0
2,800.0,4.00,0.0,0.0,0.0,1.0
3,640.0,3.19,0.0,0.0,1.0,1.0
4,520.0,2.93,0.0,0.0,1.0,1.0


#### 4.2 Fit the model

In [229]:
#X_sm = sm.add_constant(train_cols)
log_lm = sm.Logit(y, train_data)
loglm_output = log_lm.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [230]:
loglm_output.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 07 Oct 2016   Pseudo R-squ.:                 0.08166
Time:                        22:45:18   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
==============================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [231]:
loglm_output_table = np.exp(pd.concat([loglm_output.conf_int(),loglm_output.params], axis = 1))
loglm_output_table.columns = ['2.5%', '97.5%', 'OR']

In [232]:
print loglm_output_table

                2.5%     97.5%        OR
gre         1.000074  1.004372  1.002221
gpa         1.136120  4.183113  2.180027
prestige_2  0.272168  0.942767  0.506548
prestige_3  0.133377  0.515419  0.262192
prestige_4  0.093329  0.479411  0.211525
intercept   0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: Going to a Prestige 2 school has no effect on admitance.

#### 4.6 Interpret the OR of GPA

Answer: The higher your GPA the higher the odds of being admitted.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [222]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [223]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [224]:
combos.columns = ['gre','gpa','prestige', 'intercept']
combos.head(n=2)

,gre,gpa,prestige,intercept
0,220.0,2.26,1.0,1.0
1,220.0,2.26,2.0,1.0


#### 5.1 Recreate the dummy variables

In [233]:
# recreate the dummy variables
prestige_dummies = pd.get_dummies(combos['prestige'], prefix = 'prestige')
prestige_dummies.columns = ['prestige_1','prestige_2','prestige_3', 'prestige_4']
# keep only what we need for making predictions
cols_keep = ['gre', 'gpa','prestige','intercept']
combos = combos[cols_keep].join(prestige_dummies.ix[:,1:])
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,1.0,1.0,0.0,0.0,0.0
1,220.0,2.260000,2.0,1.0,1.0,0.0,0.0
2,220.0,2.260000,3.0,1.0,0.0,1.0,0.0
3,220.0,2.260000,4.0,1.0,0.0,0.0,1.0
4,220.0,2.453333,1.0,1.0,0.0,0.0,0.0


#### 5.2 Make predictions on the enumerated dataset

In [238]:
combos['admit_predict'] = loglm_output.predict(combos[train_cols])
combos.tail()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4,admit_predict
395,800.0,3.806667,4.0,1.0,0.0,0.0,1.0,0.334286
396,800.0,4.000000,1.0,1.0,0.0,0.0,0.0,0.734040
397,800.0,4.000000,2.0,1.0,1.0,0.0,0.0,0.582995
398,800.0,4.000000,3.0,1.0,0.0,1.0,0.0,0.419833
399,800.0,4.000000,4.0,1.0,0.0,0.0,1.0,0.368608


#### 5.3 Interpret findings for the last 4 observations

Answer: The last 4 predictions follow the logic from early that prestige of undergrad insititution has an affect on probability of being accepted. The last four rows hold GRE and GPA constant with only the prestige of the institution changing. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.